In [ ]:
import pandas as pd

import matplotlib.pylab as plt
import numpy as np

In [ ]:
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
topdir= "./"

filename = 'DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000_BNC.pkl'

df_sp = pd.read_parquet(f"{topdir}{filename}")

In [ ]:
df_sp

In [ ]:
df_sp.info()

In [ ]:
n_org= 100000
mask = df_sp["spmode"]== "-999"
num= len(df_sp[mask])
num

In [ ]:
df_sp[mask]

In [ ]:
len(df_sp)

In [ ]:
df_sp['spmode'].value_counts()

In [ ]:
def sig_efficiency(df,thresh,BNC= False, orig_num=None, disp= True):
    
    
    mask = df["spmode"]== "-999"
    
    df_sig= df[mask]

    if orig_num is None:
        print("Please pass in the original number of events!")
        return None
    
    num_reco= len(df_sig)

    eff_reco= num_reco / orig_num
    unc_reco= 1/(orig_num)*np.sqrt(num_reco*(1-eff_reco))
    
    cm1= df_sig["cut_-1"]== True
    cm2= df_sig["cut_2"]== True
    cm3= df_sig["cut_3"]== True
    cm4= df_sig["cut_4"]== True
    cm6= df_sig["cut_6"]== True

    not_used_in_training = df_sig['used_in_sig_train']

    num_reco_not_training = len(df_sig[not_used_in_training])
    
    thresh_mask= df_sig["proba"]>= thresh

    mask = thresh_mask & not_used_in_training
    if BNC:
        mask = mask & cm2 & cm3 & cm4
    else:
        mask = mask & cm1

    n_reco_not_training_pass_selection_criteria= len(df_sig[mask])
    
    eff_sc= n_reco_not_training_pass_selection_criteria/num_reco_not_training
    unc_sc= 1/(num_reco_not_training)*np.sqrt(n_reco_not_training_pass_selection_criteria*(1-eff_sc))
    
    unc_tot= np.sqrt(unc_sc**2+unc_reco**2)

    if disp: 
        print(f"eff (reco):   {100*eff_reco:.2f}%")
        print(f"eff (sc):     {100*eff_sc:.2f}%")
        print(f"eff (tot):    {100*eff_reco*eff_sc:.2f}%")
        print(f"unc (tot):    {100*unc_tot:.2f}%")
        print(f"% unc reco:   {100*unc_reco/eff_reco:.2f}%")        
        print(f"% unc sc:     {100*unc_sc/eff_sc:.2f}%")

        
    
    return eff_reco, eff_sc, eff_reco*eff_sc, unc_tot

In [ ]:
er, esc, etot, unc = sig_efficiency(df_sp, 0.97, BNC= True, orig_num=100000)




In [ ]:
xpts = []
ypts = []
for i in np.linspace(0,1,100):
    er, esc, etot, unc = sig_efficiency(df_sp, i, BNC= True, orig_num=100000, disp= False)
    xpts.append(i)
    ypts.append(etot)
    



In [ ]:
plt.figure()
plt.plot(xpts, ypts)
plt.xlabel("Threshold")
plt.ylabel("Efficiency")

# Playing around with uncertainies

In [ ]:
k = 90
N = 900

eff = k/N

eff_unc = (1/N)*np.sqrt(k*(1-k/N))

print(f"{100*eff:.4f} +/- {100*eff_unc:.4f}   ({eff_unc/eff:.4f} frac uncert)")

# Calculate the branching fraction

Given the number of observed events, and the threshold cut, calculate the branching fraction.

In [ ]:
def quad(x):
    sum= 0 
    for i in x:
        sum+= i**2
    return np.sqrt(sum)


In [ ]:
nB= 460e6 # 0.6%
nB_frac_unc= 0.006

n_obs= 3.5
n_obs_unc= 1.5
n_obs_frac_unc= n_obs_unc/n_obs

er, esc, etot, unc = sig_efficiency(df_sp, 0.97, BNC= True, orig_num=100000, disp= False)
eff_frac_unc= unc/etot

Br_lam_p_pi= 0.641
# This is the uncertainty
Br_lam_p_pi_unc= 0.005
# This would then be the fractional uncertainty
Br_lam_p_pi_frac_unc= Br_lam_p_pi_unc / Br_lam_p_pi


Br_B_p_lam= n_obs / (nB * eff * Br_lam_p_pi) 

# When we add the other uncertainties in quadrature we get
# back is the *fractional uncertainty* on the branching fraction
blam_frac_unc= quad([nB_frac_unc, n_obs_frac_unc, eff_frac_unc, Br_lam_p_pi_frac_unc])

blam_unc = Br_B_p_lam * blam_frac_unc

print(f"Br_B_p_lam:  {Br_B_p_lam:.3e}")
print(f"Br_frac_unc: {blam_frac_unc:.3e}")
print(f"Br_unc:      {blam_unc:.3e}")
print()

print(f"Br: {Br_B_p_lam:.1e} +/- {blam_unc:.1e}")

print(f"Br: ({1e7*Br_B_p_lam:.1f} +/- {1e7*blam_unc:.1f}) x 10^(-7)")


#print(frac)

In [ ]:
#n_obs= nB * Br_B_p_lam * eff * Br_lam_p_pi


# We want this
#Br_B_p_lam= n_obs / (nB * eff * Br_lam_p_pi) 

In [ ]:
### Total Uncertainty 

B_unc= .006
Br_lam_ppi_unc= .005
Br_B_plam_unc= 
obs_unc= #######
eff_unc= unc_tot

total_uncertainty= quad([])

In [ ]:
1/2.4